# 학습용 이미지로 데이터셋 생성
- 학습용 이미지 폴더를 순회
- 이미지 세로 픽셀별 가로줄 RGB 값 구해서 `'R'`, `'G'`, `'B'` 열을 가지는 데이터프레임 생성
- 각 이미지마다 문자 있는 영역과 없는 영역 라벨링
- `total_df`에 추가

=> 최종적으로 이미지 가로줄의 `'R'`, `'G'`, `'B'`값과 문자 여부가 매핑된 `total_df` 데이터프레임이 완성됨

In [1]:
from image_function import *
# 이미 만들어진 데이터가 있는 경우 그 위에 쌓아서 추가
# 학습 이미지 중에 이미 학습한 이미지 있는지 확인해야함! or 지워버리던가
prev_df_path = "RGB_data.csv"
if os.path.exists(prev_df_path):
    total_df = pd.read_csv(prev_df_path)
else:
    total_df = pd.DataFrame(columns=['R', 'G', 'B'])

In [2]:
train_image_list = os.listdir('train_image')
for image in train_image_list:
    # 이미지 폴더안의 이미지 경로
    image_path=os.path.join('train_image', image)
    # 이미지 픽셀 RGB값으로 데이터프레임 생성
    low_pix_mean=RGB_extract(image_path)
    data = pd.DataFrame(low_pix_mean, columns=['R','G','B'])
    # 이미지 자르기
    slice_image_vertically(image_path, 'image', 2000)
    # ocr 수행
    result=OCR_image('image', 'result_image',2000)
    # 글자 영역 추출
    boundary=[]
    for pair in result:
        top= int(min([pair[1][i][1] for i in range(4)]))+3
        bottom=int(max([pair[1][i][1] for i in range(4)]))-3
        boundary.append((top,bottom))
    # 글자 영역에 맞춰서 타겟값 매핑
    data['char']=0
    for pair in boundary:
        start, end = pair
        data.iloc[start:end+1]['char']=1
    total_df=pd.concat([total_df, data], ignore_index=True)
    print(total_df.tail())
    print("총 데이터 수:",len(total_df))

    # 학습에 사용한 이미지는 삭제 (중복 방지)
    if os.path.isfile(image_path):
        os.unlink(image_path)
    elif os.path.isdir(image_path):
        shutil.rmtree(image_path)
    

(900, 22042)
[2024/03/30 23:43:26] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\KHU/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\KHU/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e

In [3]:
total_df.to_csv("RGB_data.csv", index=False)